In [89]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time

In [148]:
data = pd.read_csv('//Users/tiagoornelas/Documents/Ironhack/Projects/Final project/scraping_cleaning/films_clean.csv')
data = data.drop('Unnamed: 0', axis=1)


In [151]:
data.rename(columns={'0': 'description'}, inplace=True)


In [153]:
def unique_values (row):

    return sorted(set(row))

data['unique'] = data['description'].apply(unique_values)

In [154]:
vocab_list = list(data['unique'])

unique_list = []

for i in vocab_list:
    for x in i:
        unique_list.append(x)

unique_vocab = sorted(set(unique_list))
vocab_list

[[' ',
  ',',
  '.',
  '1',
  '2',
  '7',
  'D',
  'H',
  'T',
  'a',
  'b',
  'c',
  'e',
  'f',
  'g',
  'h',
  'i',
  'l',
  'n',
  'o',
  'p',
  'r',
  's',
  't',
  'u',
  'w',
  'y'],
 [' ',
  "'",
  ',',
  '.',
  ':',
  'A',
  'L',
  'N',
  'O',
  'a',
  'b',
  'c',
  'd',
  'e',
  'f',
  'g',
  'h',
  'i',
  'l',
  'm',
  'n',
  'o',
  'p',
  'r',
  's',
  't',
  'u',
  'v',
  'w'],
 [' ',
  "'",
  '-',
  '.',
  'A',
  'C',
  'I',
  'W',
  'a',
  'b',
  'c',
  'd',
  'e',
  'f',
  'g',
  'h',
  'i',
  'l',
  'm',
  'n',
  'o',
  'p',
  'r',
  's',
  't',
  'u',
  'w',
  'y'],
 [' ',
  ',',
  '.',
  '3',
  '5',
  '6',
  'L',
  'M',
  'S',
  'a',
  'b',
  'c',
  'd',
  'e',
  'f',
  'g',
  'h',
  'i',
  'k',
  'l',
  'm',
  'n',
  'o',
  'p',
  'r',
  's',
  't',
  'u',
  'v',
  'w',
  'x',
  'y'],
 [' ',
  "'",
  '.',
  'L',
  'M',
  'N',
  'a',
  'b',
  'c',
  'd',
  'e',
  'g',
  'h',
  'i',
  'j',
  'k',
  'l',
  'm',
  'n',
  'o',
  'p',
  'r',
  's',
  't',
  'u',
  'v',
  

In [155]:
## vectorizing text

def char_encode (row):

    return tf.strings.unicode_split(row, input_encoding='UTF-8')

char_encode_df = data['description'].apply(char_encode)


In [156]:
# converting tokens to character ID's

ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(unique_vocab), mask_token=None)

In [157]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [158]:
def vectors_to_char_id (row):

    return ids_from_chars(tf.strings.unicode_split(row, 'UTF-8'))

all_ids = data['description'].apply(vectors_to_char_id)


all_ids_df_list = list(all_ids)
all_ids_list = []

for i in all_ids_df_list:
    for x in i:
        all_ids_list.append(x)


In [159]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids_list)

In [161]:
seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

In [162]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [163]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [164]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

In [165]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [166]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [167]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

Training Model

In [168]:
# compiling model with optimizer & loss function

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

EPOCHS = 5

model.compile(optimizer='adam', loss=loss)

In [170]:
## saving checkpoints

checkpoint_dir = './training_checkpoints_films_2'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [171]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/5
224/475 [=============>................] - ETA: 16:45 - loss: 2.4952

In [114]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [129]:
film_model = OneStep(model, chars_from_ids, ids_from_chars)

In [134]:
start = time.time()
states = None
next_char = tf.constant(['Daniel'])
result = [next_char]

for n in range(500):
  next_char, states = film_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Daniel larusso and johnny lawrence find themselves martial-arts rivals againjennifer walters navigates the complicated life of a single, 30-something attorney who also happens to be a green 6-foot-7-inch superpowered hulkthree strangers who share an obsession with true crime suddenly find themselves caught up in onefive assassins aboard a fast moving bullet train find out their missions have something in commona hard-working, blue-collar dad just wants to provide a good life for his quick-witted 10-ye 

________________________________________________________________________________

Run time: 1.5229229927062988


In [123]:
tf.saved_model.save(film_model, 'film_generator_model')

INFO:tensorflow:Assets written to: film_generator_model/assets


INFO:tensorflow:Assets written to: film_generator_model/assets
